In [ ]:
# Install required packages
!pip install accelerate
!pip install git+https://github.com/huggingface/transformers -q
!pip install torch torchaudio -q
!pip install seaborn-image
!pip install langdetect
!pip install -U deep-translator
!pip install rouge
!pip install pymorphy2
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os
import time
import cv2
import tempfile
import numpy as np
import pandas as pd
import moviepy.editor as mp
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans as SklearnKMeans
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration
from langdetect import detect
import torch
from deep_translator import GoogleTranslator
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import pymorphy2


In [ ]:
# Initialize models
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()
whisper = pipeline('automatic-speech-recognition', model='openai/whisper-tiny', device=0 if torch.cuda.is_available() else -1)
t5_model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-base-absum')
t5_tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-base-absum')
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# Function to translate text using deep_translator library
def translate_text(text, src_lang='auto', dest_lang='ru'):
    try:
        translator = GoogleTranslator(source=src_lang, target=dest_lang)
        translated_text = translator.translate(text)
    except Exception as e:
        print(f"Translation error: {e}")
        translated_text = text  # Fallback to original text in case of an error
    return translated_text

# Function to transcribe audio segment from video
def transcribe_audio_segment(video_file_path, start_time, end_time):
    #print("Loading video and extracting audio segment...")
    video = mp.VideoFileClip(video_file_path)
    video_segment = video.subclip(start_time, end_time)

    #print("Writing audio segment to temporary file...")
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio_file:
        audio_path = temp_audio_file.name
        video_segment.audio.write_audiofile(audio_path, codec='pcm_s16le')

    #print("Transcribing audio segment...")
    audio_text = whisper(audio_path)['text']

    # Clean up temporary audio file
    os.remove(audio_path)

    # Detect language of the transcribed text
    audio_language = detect(audio_text)

    return audio_text, audio_language

# Your text summarization function
def summarize_text_t5(text, max_length=100, num_beams=4, repetition_penalty=2.5):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    t5_model.to(device)
    input_ids = t5_tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)

    with torch.no_grad():
        summary_ids = t5_model.generate(input_ids, max_length=max_length, num_beams=num_beams, repetition_penalty=repetition_penalty)

    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Your caption generation function
def generate_captions(frames, processor, model):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    captions = []
    for frame in frames:
        inputs = processor(frame, return_tensors="pt").to(device)
        with torch.no_grad():
            out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    return captions

# Your keyframe selection function
def select_keyframes_with_captions(video_path, clusters=5, num_keyframes=6):
    start_time = time.time()
    print("Loading video...")
    model_name = "Salesforce/blip-image-captioning-base"
    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForConditionalGeneration.from_pretrained(model_name)
    reader = cv2.VideoCapture(video_path)

    frames = []
    frame_count = 0
    #print("Reading frames...")
    while True:
        ret, frame = reader.read()
        if ret:
            if frame_count % 4 == 0:
                frames.append(frame)
            frame_count += 1
        else:
            break
    reader.release()

    #print(f"Total frames read: {frame_count}")
    #print(f"Total frames appended: {len(frames)}")

    if len(frames) == 0:
        print("No frames read from the video file. Exiting function.")
        return [], [], []

    #print("Downsizing frames...")
    frames_downsized = [cv2.resize(frame, (frame.shape[1] // 4, frame.shape[0] // 4)) for frame in frames]
    frames_downsized = np.array(frames_downsized)
    frames_downsized_flattened = frames_downsized.reshape(frames_downsized.shape[0], -1)

    #print("Applying PCA...")
    pca_start_time = time.time()
    pca = PCA(n_components=2)
    frames_2dim = pca.fit_transform(frames_downsized_flattened)
    #print(f"PCA done in {time.time() - pca_start_time:.2f} seconds")

    #print("Applying KMeans clustering...")
    kmeans_start_time = time.time()
    kmeans = SklearnKMeans(n_clusters=clusters, random_state=0)
    kmeans.fit(frames_2dim)
    #print(f"KMeans clustering done in {time.time() - kmeans_start_time:.2f} seconds")

    labels = kmeans.labels_
    centroid_positions = kmeans.cluster_centers_

    #print("Selecting keyframes...")
    label_frame_indices = []
    for i in range(clusters):
        input_data = frames_2dim[labels == i]
        input_indices = np.argwhere(labels == i)
        closest_index = np.argmin(np.abs(input_data - centroid_positions[i]).sum(axis=1))
        abs_index = input_indices[closest_index]
        label_frame_indices.append(abs_index[0])

    selected_frames = [frames[idx] for idx in label_frame_indices]
    selected_indices = label_frame_indices[:num_keyframes]

    #print(f"Selected {len(selected_frames)} keyframes")

    #print("Generating captions...")
    captions_start_time = time.time()
    selected_captions = generate_captions(selected_frames, processor, model)
    #print(f"Captions generated in {time.time() - captions_start_time:.2f} seconds")

    #print(f"Total keyframe selection and caption generation done in {time.time() - start_time:.2f} seconds")

    return selected_frames, selected_captions, selected_indices


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn')
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

# Necessary imports
import pandas as pd
import os
import gc
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import wordnet
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize



# Function to detect language
def detect_language(text):
    if all('а' <= c <= 'я' or 'А' <= c <= 'Я' for c in text):
        return 'ru'
    else:
        return 'en'

# Function to get wordnet POS
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Function to lemmatize text
def lemmatize_text(text, language='en'):
    if language == 'en':
        return ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in nltk.word_tokenize(text)])
    elif language == 'ru':
        return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
    else:
        return text

# Function to extract keywords
def extract_keywords(text, language='en', top_n=10):
    vectorizer = CountVectorizer(max_features=top_n, stop_words='english' if language == 'en' else None)
    X = vectorizer.fit_transform([text])
    keywords = vectorizer.get_feature_names_out()
    return keywords

# Function to compute ROUGE score
def compute_rouge_score(hypothesis, reference):
    rouge = Rouge()
    lang_hypothesis = detect_language(hypothesis)
    lang_reference = detect_language(reference)
    hypothesis = lemmatize_text(hypothesis, lang_hypothesis)
    reference = lemmatize_text(reference, lang_reference)
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]['rouge-l']['f']

# Function to compute similarity
def compute_similarity(text1, text2):
    lang_text1 = detect_language(text1)
    lang_text2 = detect_language(text2)
    text1 = lemmatize_text(text1, lang_text1)
    text2 = lemmatize_text(text2, lang_text2)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# Function to compute keyword similarity
def compute_keyword_similarity(text1, text2, top_n=10):
    lang_text1 = detect_language(text1)
    lang_text2 = detect_language(text2)
    text1_keywords = extract_keywords(text1, lang_text1, top_n)
    text2_keywords = extract_keywords(text2, lang_text2, top_n)
    intersection = len(set(text1_keywords) & set(text2_keywords))
    union = len(set(text1_keywords) | set(text2_keywords))
    return intersection / union if union != 0 else 0

# Main function to generate time framed summarization
def generate_time_framed_summarization(video_path, num_keyframes=6):
    # Select keyframes with captions
    selected_frames, captions, frame_indices = select_keyframes_with_captions(video_path, num_keyframes=num_keyframes)

    if len(selected_frames) == 0:
        print("No frames were selected. Exiting.")
        return None

    combined_texts = []
    timestamps = []
    intervals = []
    transcribed_texts = []
    image_captions = []

    for idx, frame_index in enumerate(frame_indices):
        timestamp = frame_index // 30  # Assuming video FPS is 30
        timestamps.append(f"{frame_index} ({timestamp}s)")

        start_time = max(0, timestamp - 10)  # Start 10 seconds before the timestamp
        end_time = timestamp + 10  # End 10 seconds after the timestamp

        audio_clip_text, audio_language = transcribe_audio_segment(video_path, start_time, end_time)
        transcribed_texts.append(audio_clip_text)

        # Translate the transcribed text to English if not already in English
        if audio_language != 'en':
            translation = translate_text(audio_clip_text, src_lang=audio_language, dest_lang='en')
            audio_clip_text = translation

        # Translate the caption to Russian
        caption_russian = translate_text(captions[idx], src_lang='en', dest_lang='ru')

        prompt = f"Объекты на видео: {caption_russian}. Аудио из видео: {audio_clip_text}"
        summarized_text = summarize_text_t5(prompt)
        combined_texts.append(summarized_text)

        image_captions.append(caption_russian)

        interval = f"{start_time}-{end_time}"
        intervals.append(interval)

    df = pd.DataFrame({
        'Frame Number (Timestamp)': timestamps,
        'Interval (seconds)': intervals,
        'Transcribed Text': transcribed_texts,
        'Combined Text': combined_texts,
        'Image Caption': image_captions
    })

    return df

# Function to compare summarizations with YandexGPT and description
def compare_summarizations_with_yandexGPT_and_description(video_name, df, custom_dataset):
    # Find corresponding row in the custom dataset
    video_row = custom_dataset[custom_dataset['video_name'] == video_name]

    if video_row.empty:
        print(f"Video name {video_name} not found in the custom dataset.")
        return None, None, None

    # Load YandexGPT summarization text
    yandex_summary_path = f"/AutogenerateVideoDescriptions/CustomDataset/{video_row['stt_name'].iloc[0]}"
    with open(yandex_summary_path, 'r') as file:
        yandex_summary_text = file.read()

    # Author description
    author_description = video_row['description'].iloc[0]

    # Combine all 'Combined Text' from df into one string
    pipeline_summary = " ".join(df['Combined Text'].tolist())

    return yandex_summary_text, author_description, pipeline_summary

# Function to evaluate dataset
def evaluate_dataset(video_folder_path, custom_dataset, output_file="evaluation_results.csv", batch_size=5):
    results = []
    video_files = [f for f in os.listdir(video_folder_path) if f.endswith('.mp4')]
    total_videos = len(video_files)

    for batch_start in range(0, total_videos, batch_size):
        batch_end = min(batch_start + batch_size, total_videos)
        current_batch = video_files[batch_start:batch_end]

        for idx, video_name in enumerate(current_batch):
            print(f"Starting evaluation of video {batch_start + idx + 1}/{total_videos}: {video_name}")
            video_path = os.path.join(video_folder_path, video_name)

            try:
                df = generate_time_framed_summarization(video_path)
                if df is None:
                    print("Summarization failed.")
                    continue

                print("Summarization successful.")

                # Compare summarizations with YandexGPT and author description
                result = compare_summarizations_with_yandexGPT_and_description(video_name, df, custom_dataset)
                if result is None:
                    print(f"Skipping video {video_name} due to missing data.")
                    continue

                yandex_summary, author_description, pipeline_summary = result

                # Compute metrics
                rouge_score = compute_rouge_score(pipeline_summary, yandex_summary)
                similarity_author = compute_similarity(pipeline_summary, author_description)
                similarity_yandex = compute_similarity(pipeline_summary, yandex_summary)
                keyword_similarity_yandex = compute_keyword_similarity(pipeline_summary, yandex_summary)

                results.append({
                    'Video Name': video_name,
                    'ROUGE Score': rouge_score,
                    'Similarity with Author Description': similarity_author,
                    'Similarity with YandexGPT': similarity_yandex,
                    'Keyword Similarity with YandexGPT': keyword_similarity_yandex
                })

                print(f"Evaluation of video {batch_start + idx + 1}/{total_videos} complete.")

            except Exception as e:
                print(f"Error processing video {video_name}: {e}")

            # Free up memory
            del df
            gc.collect()

        # Save intermediate results
        if results:
            results_df = pd.DataFrame(results)
            results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            results = []  # Clear results list to free memory

    # Final save in case there are remaining results
    if results:
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

    return pd.read_csv(output_file)  # Load the final results





In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn')
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

# Necessary imports
import pandas as pd
import os
import gc
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import wordnet
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

# Initialize lemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()
morph = MorphAnalyzer()

# Function to detect language
def detect_language(text):
    if all('а' <= c <= 'я' or 'А' <= c <= 'Я' for c in text):
        return 'ru'
    else:
        return 'en'

# Function to get wordnet POS
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Function to lemmatize text
def lemmatize_text(text, language='en'):
    if language == 'en':
        return ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in nltk.word_tokenize(text)])
    elif language == 'ru':
        return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
    else:
        return text

# Function to extract keywords
def extract_keywords(text, language='en', top_n=10):
    vectorizer = CountVectorizer(max_features=top_n, stop_words='english' if language == 'en' else None)
    X = vectorizer.fit_transform([text])
    keywords = vectorizer.get_feature_names_out()
    return keywords

# Function to compute ROUGE score
def compute_rouge_score(hypothesis, reference):
    rouge = Rouge()
    lang_hypothesis = detect_language(hypothesis)
    lang_reference = detect_language(reference)
    hypothesis = lemmatize_text(hypothesis, lang_hypothesis)
    reference = lemmatize_text(reference, lang_reference)
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]['rouge-l']['f']

# Function to compute similarity
def compute_similarity(text1, text2):
    lang_text1 = detect_language(text1)
    lang_text2 = detect_language(text2)
    text1 = lemmatize_text(text1, lang_text1)
    text2 = lemmatize_text(text2, lang_text2)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# Function to compute keyword similarity
def compute_keyword_similarity(text1, text2, top_n=10):
    lang_text1 = detect_language(text1)
    lang_text2 = detect_language(text2)
    text1_keywords = extract_keywords(text1, lang_text1, top_n)
    text2_keywords = extract_keywords(text2, lang_text2, top_n)
    intersection = len(set(text1_keywords) & set(text2_keywords))
    union = len(set(text1_keywords) | set(text2_keywords))
    return intersection / union if union != 0 else 0

# Main function to generate time framed summarization
def generate_time_framed_summarization(video_path, num_keyframes=6):
    # Select keyframes with captions
    selected_frames, captions, frame_indices = select_keyframes_with_captions(video_path, num_keyframes=num_keyframes)

    if len(selected_frames) == 0:
        print("No frames were selected. Exiting.")
        return None

    combined_texts = []
    timestamps = []
    intervals = []
    transcribed_texts = []
    image_captions = []

    for idx, frame_index in enumerate(frame_indices):
        timestamp = frame_index // 30  # Assuming video FPS is 30
        timestamps.append(f"{frame_index} ({timestamp}s)")

        start_time = max(0, timestamp - 10)  # Start 10 seconds before the timestamp
        end_time = timestamp + 10  # End 10 seconds after the timestamp

        audio_clip_text, audio_language = transcribe_audio_segment(video_path, start_time, end_time)
        transcribed_texts.append(audio_clip_text)

        # Translate the transcribed text to English if not already in English
        if audio_language != 'en':
            translation = translate_text(audio_clip_text, src_lang=audio_language, dest_lang='en')
            audio_clip_text = translation

        # Translate the caption to Russian
        caption_russian = translate_text(captions[idx], src_lang='en', dest_lang='ru')

        prompt = f"Объекты на видео: {caption_russian}. Аудио из видео: {audio_clip_text}"
        summarized_text = summarize_text_t5(prompt)
        combined_texts.append(summarized_text)

        image_captions.append(caption_russian)

        interval = f"{start_time}-{end_time}"
        intervals.append(interval)

    df = pd.DataFrame({
        'Frame Number (Timestamp)': timestamps,
        'Interval (seconds)': intervals,
        'Transcribed Text': transcribed_texts,
        'Combined Text': combined_texts,
        'Image Caption': image_captions
    })

    return df

# Function to compare summarizations with YandexGPT and description
def compare_summarizations_with_yandexGPT_and_description(video_name, df, custom_dataset):
    # Find corresponding row in the custom dataset
    video_row = custom_dataset[custom_dataset['video_name'] == video_name]

    if video_row.empty:
        print(f"Video name {video_name} not found in the custom dataset.")
        return None, None, None

    # Load YandexGPT summarization text
    yandex_summary_path = f"/AutogenerateVideoDescriptions/CustomDataset/{video_row['stt_name'].iloc[0]}"
    with open(yandex_summary_path, 'r') as file:
        yandex_summary_text = file.read()

    # Author description
    author_description = video_row['description'].iloc[0]

    # Combine all 'Combined Text' from df into one string
    pipeline_summary = " ".join(df['Combined Text'].tolist())

    return yandex_summary_text, author_description, pipeline_summary




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Function to evaluate dataset
def evaluate_dataset(video_folder_path, custom_dataset, output_file="evaluation_results.csv", batch_size=5):
    results = []
    video_files = [f for f in os.listdir(video_folder_path) if f.endswith('.mp4')]
    total_videos = len(video_files)

    for batch_start in range(0, total_videos, batch_size):
        batch_end = min(batch_start + batch_size, total_videos)
        current_batch = video_files[batch_start:batch_end]

        for idx, video_name in enumerate(current_batch):
            print(f"Starting evaluation of video {batch_start + idx + 1}/{total_videos}: {video_name}")
            video_path = os.path.join(video_folder_path, video_name)

            try:
                df = generate_time_framed_summarization(video_path)
                if df is None:
                    print("Summarization failed.")
                    continue

                print("Summarization successful.")

                # Compare summarizations with YandexGPT and author description
                result = compare_summarizations_with_yandexGPT_and_description(video_name, df, custom_dataset)
                if result is None:
                    print(f"Skipping video {video_name} due to missing data.")
                    continue

                yandex_summary, author_description, pipeline_summary = result

                # Compute metrics
                rouge_score = compute_rouge_score(pipeline_summary, author_description)  # Use author description as gold standard
                similarity_author = compute_similarity(pipeline_summary, author_description)
                similarity_yandex = compute_similarity(pipeline_summary, yandex_summary)
                keyword_similarity_yandex = compute_keyword_similarity(pipeline_summary, yandex_summary)

                # Compute text lengths
                len_pipeline_summary = len(lemmatize_text(pipeline_summary))
                len_author_description = len(lemmatize_text(author_description))
                len_yandex_summary = len(lemmatize_text(yandex_summary))

                results.append({
                    'Video Name': video_name,
                    'ROUGE Score': rouge_score,
                    'Similarity with Author Description': similarity_author,
                    'Similarity with YandexGPT': similarity_yandex,
                    'Keyword Similarity with YandexGPT': keyword_similarity_yandex,
                    'Pipeline Summary Length': len_pipeline_summary,
                    'Author Description Length': len_author_description,
                    'Yandex Summary Length': len_yandex_summary
                })

                print(f"Evaluation of video {batch_start + idx + 1}/{total_videos} complete.")

            except Exception as e:
                print(f"Error processing video {video_name}: {e}")

            # Free up memory
            del df
            gc.collect()

        # Save intermediate results
        if results:
            results_df = pd.DataFrame(results)
            results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            results = []  # Clear results list to free memory

    # Final save in case there are remaining results
    if results:
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

    return pd.read_csv(output_file)  # Load the final results


In [ ]:
# Path to your dataset folder
video_folder_path = "/AutogenerateVideoDescription/CustomDataset"
custom_df = pd.read_csv('custom_dataset.csv')

# Evaluate the dataset
evaluation_results = evaluate_dataset(video_folder_path, custom_df)

# Display the results
print(evaluation_results)


Starting evaluation of video 1/32: 316.mp4
Loading video...
MoviePy - Writing audio in /tmp/tmpmptqhgxb.wav


MoviePy - Done.


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


MoviePy - Writing audio in /tmp/tmpx57lzn6f.wav


MoviePy - Done.
MoviePy - Writing audio in /tmp/tmp0y37hpk1.wav


...

MoviePy - Done.
MoviePy - Writing audio in /tmp/tmpcpz2nxg1.wav


MoviePy - Done.
MoviePy - Writing audio in /tmp/tmp7nyqlcwj.wav


MoviePy - Done.
MoviePy - Writing audio in /tmp/tmpeut4y3na.wav


MoviePy - Done.


Summarization successful.
Evaluation of video 32/32 complete.
   Video Name  ROUGE Score  Similarity with Author Description  \
0     316.mp4     0.187500                            0.020037   
1      13.mp4     0.108108                            0.067827   
2     173.mp4     0.145833                            0.047665   
3     102.mp4     0.126984                            0.000000   
4     122.mp4     0.117647                            0.000000   
5     175.mp4     0.037037                            0.000000   
6     135.mp4     0.101266                            0.010354   
7     185.mp4     0.045455                            0.031984   
8     208.mp4     0.101266                            0.045229   
9     112.mp4     0.140351                            0.054929   
10    270.mp4     0.027778                            0.010245   
11    290.mp4     0.116505                            0.045943   
12    413.mp4     0.105263                            0.038550   
13    436.mp4 

In [ ]:
evaluation_results

,Video Name,ROUGE Score,Similarity with Author Description,Similarity with YandexGPT,Keyword Similarity with YandexGPT,Pipeline Summary Length,Author Description Length,Yandex Summary Length
0,316.mp4,0.187500,0.020037,0.036996,0.052632,348,239,860
1,13.mp4,0.108108,0.067827,0.084796,0.111111,294,244,1053
2,173.mp4,0.145833,0.047665,0.041461,0.000000,363,359,493
3,102.mp4,0.126984,0.000000,0.021033,0.000000,206,247,944
4,122.mp4,0.117647,0.000000,0.042497,0.000000,288,206,818
5,175.mp4,0.037037,0.000000,0.021695,0.000000,241,219,1126
6,135.mp4,0.101266,0.010354,0.006062,0.000000,238,359,904
7,185.mp4,0.045455,0.031984,0.057825,0.052632,227,64,888
8,208.mp4,0.101266,0.045229,0.014838,0.000000,377,200,611
9,112.mp4,0.140351,0.054929,0.032719,0.000000,257,195,1834


In [ ]:
# Select the desired columns
selected_columns = ['ROUGE Score', 'Similarity with Author Description',
                    'Similarity with YandexGPT', 'Keyword Similarity with YandexGPT',
                    'Pipeline Summary Length', 'Author Description Length', 'Yandex Summary Length']

# Calculate statistics for each selected column
worst_values = evaluation_results[selected_columns].min()
average_values = evaluation_results[selected_columns].mean()
best_values = evaluation_results[selected_columns].max()

# Create a DataFrame to store the statistics
statistics_df = pd.DataFrame({
    'Feature': selected_columns,
    'Worst': worst_values.values,
    'Average': average_values.values,
    'Best': best_values.values
})

# Display the statistics table
print(statistics_df)


                              Feature       Worst      Average         Best
0                         ROUGE Score    0.027778     0.095125     0.187500
1  Similarity with Author Description    0.000000     0.024012     0.098623
2           Similarity with YandexGPT    0.000000     0.033688     0.106914
3   Keyword Similarity with YandexGPT    0.000000     0.021747     0.111111
4             Pipeline Summary Length  167.000000   245.593750   377.000000
5           Author Description Length   64.000000   255.843750   525.000000
6               Yandex Summary Length  493.000000  1094.843750  1834.000000


In [ ]:
# Transpose the statistics DataFrame
statistics_df_transposed = statistics_df.transpose()

# Save the transposed DataFrame to a CSV file
statistics_df_transposed.to_csv("statistics.csv", index=False)

# Display the transposed statistics table
print(statistics_df_transposed)


                   0                                   1  \
Feature  ROUGE Score  Similarity with Author Description   
Worst       0.027778                                 0.0   
Average     0.095125                            0.024012   
Best          0.1875                            0.098623   

                                 2                                  3  \
Feature  Similarity with YandexGPT  Keyword Similarity with YandexGPT   
Worst                          0.0                                0.0   
Average                   0.033688                           0.021747   
Best                      0.106914                           0.111111   

                               4                          5  \
Feature  Pipeline Summary Length  Author Description Length   
Worst                      167.0                       64.0   
Average                245.59375                  255.84375   
Best                       377.0                      525.0   

                 